In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/baselines-project/test_bart_counterspeech.csv
/kaggle/input/baselines-project/llama_fs_csv.csv
/kaggle/input/baselines-project/dialogpt_predicted_cs.csv
/kaggle/input/baselines-project/Llamazeroshot_counterspeech.csv


In [3]:
%pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=122bca4e0af8083dc22c1ad97cf281b8f59dd5afcbd56079afa09a138eb804b2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install detoxify

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from tqdm import tqdm
from detoxify import Detoxify





In [10]:

csv_paths = {

    "BART": "/kaggle/input/projecteval/test_bart_counterspeech.csv",
    "LLaMA_few_shot": "/kaggle/input/projecteval/llama_fs_csv.csv",
    "DialoGPT": "/kaggle/input/projecteval/dialogpt_predicted_cs.csv",
    "LLaMA_zero_shot": "/kaggle/input/projecteval/Llamazeroshot_counterspeech.csv"

}


gt_column = "counterspeech"     # Ground truth
pred_column = "predicted_cs"    #predicted cs column



In [11]:
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
smoothie = SmoothingFunction().method4

metrics_only_results = {}

for model_name, file_path in csv_paths.items():
    df = pd.read_csv(file_path)
    references = df[gt_column].astype(str).tolist()
    predictions = df[pred_column].astype(str).tolist()

    bleu_scores = []
    rouge_l_scores = []

    for ref, pred in tqdm(zip(references, predictions), total=len(references), desc=f"Scoring {model_name}"):
        ref_tokens = [ref.split()]
        pred_tokens = pred.split()

        bleu = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)
        bleu_scores.append(bleu)

        rouge_l = rouge.score(ref, pred)['rougeL'].fmeasure
        rouge_l_scores.append(rouge_l)

    # BERTScore
    P, R, F1 = bert_score(predictions, references, lang="en", verbose=True, device="cuda")
    bert_f1_scores = F1.tolist()

    # Save partial results
    metrics_only_results[model_name] = {
        "BLEU": sum(bleu_scores) / len(bleu_scores),
        "ROUGE-L": sum(rouge_l_scores) / len(rouge_l_scores),
        "BERTScore-F1": sum(bert_f1_scores) / len(bert_f1_scores),
    }

# Save as CSV (optional)
metrics_df = pd.DataFrame(metrics_only_results).T
metrics_df.to_csv("core_metrics_results.csv")
print("\nCore Evaluation Metrics:")
print(metrics_df)

Scoring BART: 100%|██████████| 2971/2971 [00:06<00:00, 438.18it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/93 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/47 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/93 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/47 [00:00<?, ?it/s]

done in 35.39 seconds, 83.96 sentences/sec

Core Evaluation Metrics:
                     BLEU   ROUGE-L  BERTScore-F1
BART             0.152081  0.194748      0.872589
LLaMA_few_shot   0.138153  0.168510      0.868131
DialoGPT         0.105171  0.126520      0.854352
LLaMA_zero_shot  0.135371  0.162974      0.866134


In [12]:
from detoxify import Detoxify
import pandas as pd

# Store toxicity results
toxicity_results = {}

for model_name, file_path in csv_paths.items():
    df = pd.read_csv(file_path)
    predictions = df[pred_column].astype(str).tolist()

    # Compute toxicity scores
    toxicity_scores = Detoxify('original').predict(predictions)["toxicity"]
    avg_toxicity = sum(toxicity_scores) / len(toxicity_scores)

    toxicity_results[model_name] = {"Toxicity": avg_toxicity}

# Save as CSV (optional)
toxicity_df = pd.DataFrame(toxicity_results).T
toxicity_df.to_csv("toxicity_results.csv")
print("\nToxicity Evaluation Results:")
print(toxicity_df)


Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
100%|██████████| 418M/418M [00:01<00:00, 415MB/s] 


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


Toxicity Evaluation Results:
                 Toxicity
BART             0.017560
LLaMA_few_shot   0.118123
DialoGPT         0.052261
LLaMA_zero_shot  0.014190


In [13]:
combined_df = metrics_df.join(toxicity_df)

# Save to single CSV
combined_df.to_csv("evaluation_results.csv")

print("\n📊 Final Evaluation Results (All Metrics):")
print(combined_df)


📊 Final Evaluation Results (All Metrics):
                     BLEU   ROUGE-L  BERTScore-F1  Toxicity
BART             0.152081  0.194748      0.872589  0.017560
LLaMA_few_shot   0.138153  0.168510      0.868131  0.118123
DialoGPT         0.105171  0.126520      0.854352  0.052261
LLaMA_zero_shot  0.135371  0.162974      0.866134  0.014190
